<a href="https://colab.research.google.com/github/AvinivA/DSL-project/blob/main/Basic_logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the library using pip
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

#Our sentences we like to encode
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']

#Sentences are encoded by calling model.encode()
embeddings = model.encode(sentences)

#Print the embeddings
for sentence, embedding in zip(sentences, embeddings):
    print("Sentence:", sentence)
    #print("Embedding:", embedding)
    print(len(embedding))
    print("")

Sentence: This framework generates embeddings for each input sentence
768

Sentence: Sentences are passed as a list of string.
768

Sentence: The quick brown fox jumps over the lazy dog.
768



In [ ]:
SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: XLMRobertaModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

# Try using logistic regression 

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np 
import pandas as pd 
from sentence_transformers import SentenceTransformer

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/DSL project/train_reviews.csv')

In [ ]:
train_df.head()

,review_title,review_body,review_date,review_rating,number_of_photos,helpful_vote,reviewer_ID,fake_asin,fake_review,product_ID,review_ID
0,Super product,"My husband has a hard time sleeping, the very ...",2020-04-13,5,0,0,380263,1,1,389,34510
1,Very sturdy,Rving,2020-08-29,5,0,0,845137,0,0,99,381688
2,Five Stars,has dramatically improved my toe nail appearances,2018-02-27,5,0,0,659759,0,0,107,417933
3,Great portable irrigator,I recently got braces and needed this for work...,2012-05-16,5,0,1,73493,0,0,66,166465
4,Four Stars,The Salad Shooter works great and saves time.,2015-01-05,4,0,0,714841,0,0,14,289006


In [ ]:
df = train_df[['review_title','review_body','review_rating','number_of_photos','helpful_vote','fake_review']].copy()

In [ ]:
df.dtypes

review_title        object
review_body         object
review_rating        int64
number_of_photos     int64
helpful_vote         int64
fake_review          int64
dtype: object

In [ ]:
df['review'] = df.apply(lambda x: str(x['review_title'])+ str(". ") + str(x['review_body']), axis=1 )

In [ ]:
df.head()

,review_title,review_body,review_rating,number_of_photos,helpful_vote,fake_review,review
0,Super product,"My husband has a hard time sleeping, the very ...",5,0,0,1,Super product. My husband has a hard time slee...
1,Very sturdy,Rving,5,0,0,0,Very sturdy. Rving
2,Five Stars,has dramatically improved my toe nail appearances,5,0,0,0,Five Stars. has dramatically improved my toe n...
3,Great portable irrigator,I recently got braces and needed this for work...,5,0,1,0,Great portable irrigator. I recently got brace...
4,Four Stars,The Salad Shooter works great and saves time.,4,0,0,0,Four Stars. The Salad Shooter works great and ...


In [ ]:
df.drop(['review_title','review_body'],inplace=True, axis=1)

In [ ]:
def concatenate_categorical_values_in_text(row):
  rating = row['review_rating']
  photos = row['number_of_photos']
  votes = row['helpful_vote']
  text = row['review']

  new_string = "The rating is {}. ".format(rating)
  new_string += "The number of phots is {}. ".format(photos)
  new_string += "The number of helpful votes is {} ".format(votes)
  new_string += text

  return new_string

In [ ]:
df_mod = df.copy()

In [ ]:
df_mod.head()

,review_rating,number_of_photos,helpful_vote,fake_review,review
0,5,0,0,1,Super product. My husband has a hard time slee...
1,5,0,0,0,Very sturdy. Rving
2,5,0,0,0,Five Stars. has dramatically improved my toe n...
3,5,0,1,0,Great portable irrigator. I recently got brace...
4,4,0,0,0,Four Stars. The Salad Shooter works great and ...


In [ ]:
df_mod["review"] = df_mod.apply(lambda x:concatenate_categorical_values_in_text(x),axis=1)

In [ ]:
df_mod.iloc[[0]]

,review_rating,number_of_photos,helpful_vote,fake_review,review
0,5,0,0,1,The rating is 5. The number of phots is 0. The...


In [ ]:
df_mod.drop(['review_rating','number_of_photos','helpful_vote'],axis=1,inplace=True)

In [ ]:
#Creating sentence embeddings using the sentence transformer package
sentence_transformer_model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')

In [ ]:
df_mod.rename(columns = {'review':'text'}, inplace = True)

In [ ]:
#function to transform text data
def get_paragraph_embeddings(row):
  embedding = sentence_transformer_model.encode(row["text"])
  return embedding

In [ ]:
#implementing sentence embedding using apply() function 
df_mod["embeddings"] = df_mod.apply(lambda x: get_paragraph_embeddings(x), axis=1)

In [ ]:
#df_mod = pd.read_csv('/content/drive/MyDrive/DSL project/train_reviews.csv')

In [ ]:
df_mod.head()

In [ ]:
df_mod.drop(['text'],axis=1,inplace=True)

In [ ]:
df_mod.to_csv("/content/drive/MyDrive/DSL project/embeddings.csv")

In [ ]:
import re

In [ ]:
#Cleaning the dataset to obtain the embeddings as columns of dataframe
def clean_embeddings(row):
  data = row["embeddings"]
  list_of_nums = []
  #data = re.sub(r"\[","",data)
  #data = re.sub(r"\]","",data)
  #data = list(data)
  #data = data.replace('[',"")
  #data = data.replace(']',"")
  for i in range(len(data)):
    if (data[i] != ''):
      list_of_nums.append(float(data[i]))
  #list_of_nums = data.split(" ")

  #clean_list = []
  #final_clean_list = []

  #for entry in list_of_nums:
    #if (entry != ''):
      #clean_list.append(entry)

  #final_clean_list = [float(entry) for entry in clean_list]

  return list_of_nums


In [ ]:
df_mod["embeddings"][0]

In [ ]:
#temp = df_mod["embeddings"][0]
#temp = re.sub(r"\[","",temp)
#temp = re.sub(r"\]","",temp)
#for i in range(len(temp)):
  #print(temp[i])

In [ ]:
df_mod["clean_embeddings"] = df_mod.apply(lambda x: clean_embeddings(x), axis = 1)

In [ ]:
df_mod.drop("embeddings", axis=1, inplace = True)

In [ ]:
df_mod = df_mod.rename({"clean_embeddings" : "embeddings"}, axis=1)

In [ ]:
df_embeddings_explode = pd.DataFrame(df_mod["embeddings"].to_list())
df_mod.drop("embeddings", axis=1, inplace = True)
df_embeddings_explode.fillna(0, inplace = True)
df_final = pd.concat([df_embeddings_explode, df_mod], axis = 1)

In [ ]:
#installing required packages
!pip install matplotlib # https://matplotlib.org/
!pip install -U scikit-learn # https://scikit-learn.org/
!pip install joblib # https://joblib.readthedocs.io/en/latest/

In [ ]:
#importing the installed packages
import matplotlib.pyplot as plt 
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.neural_network import MLPClassifier
import joblib

In [ ]:
#assigning independent variables into X and dependent variable to y
X = df_final.drop("fake_review", axis = 1) #embeddings as columns are independent variable
y = df_final["fake_review"] # label (0 or 1) is the dependent variable

In [ ]:
train_split = 0.7

In [ ]:
X_train, X_test_1, y_train, y_test_1 = train_test_split(df_final.drop('fake_review', axis=1), df_final['fake_review'], test_size=(1-train_split), random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test_1, y_test_1, test_size=0.333, random_state=42)
X_val_sup, X_val_unsup, y_val_sup, y_val_unsup = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

In [ ]:
# a. Logistic Regression (https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
param_grid = { 
    'C' : [2, 6, 10]
}
#Grid Search for Logistic Regression
LogReg = GridSearchCV(estimator=LogisticRegression(solver='liblinear',random_state=0), param_grid=param_grid)
LogReg.fit(X_train, y_train)   
print("Best Params: {}".format(LogReg.best_params_))
print("Best Scores: {}".format(LogReg.best_score_))   

pred = LogReg.predict(X_val_sup)
accuracy = LogReg.score(X_val_sup, y_val_sup)
print("Accuracy = {}".format(accuracy))
print("Macro F1 Score = {}".format(f1_score(y_val_sup, pred, average='macro')))
print(classification_report(y_val_sup, pred))

In [ ]:
test_data = pd.read_csv('/content/drive/MyDrive/DSL project/test_embeddings.csv')

In [ ]:
test_data["clean_embeddings"] = df_mod.apply(lambda x: clean_embeddings(x), axis = 1)
test_data.drop(["embeddings"],axis=1,inplace=True)

In [ ]:
predictions = LogReg.predict(test_data)